In [ ]:
!pip install emcee corner

In [ ]:
import corner
import emcee
import numpy as np
import scipy
import scipy.stats
import matplotlib.pyplot as plt

In [ ]:
def SimpleColorPlotFromFunc( 
    Func2D = None,
    xmin = None,
    xmax = None, 
    ymin = None,
    ymax = None, 
    ):

    #Make the list of poitns to plug in from the boundaries:
    x = np.linspace(xmin, xmax, 100)
    y = np.linspace(ymin, ymax, 100)
    X, Y = np.meshgrid(x, y)
    PointsToPlugIn = np.vstack([X.ravel(), Y.ravel()])
    PointsToPlugInDataset = PointsToPlugIn.T


    #plug in the list of points:
    FunctionResultValuesForGrid = []
    for Point in PointsToPlugInDataset:
        Value = Func2D(Point)
        FunctionResultValuesForGrid.append(Value)

    #reshape stuff in a confusing way so matplotlib can think of the data like a matrix
    Z = np.reshape(FunctionResultValuesForGrid, X.shape).T


    #Actually construct the figure...
    plt.figure()
    heatmap = plt.imshow( 
        np.rot90(Z), 
        extent=[xmin, xmax, ymin, ymax] ,
        aspect = 'auto' ,
        interpolation = None,
        )  


In [ ]:

Gaussian1Mean = [1,1]
Gaussian2Mean = [2,4]

Gaussian1Covariance = np.identity(2)*1
Gaussian2Covariance = np.identity(2)*.5


def Gaussian1(ABpoint):
    return scipy.stats.multivariate_normal.pdf( ABpoint, mean = Gaussian1Mean, 
                                               cov = Gaussian1Covariance )

def Gaussian2(ABpoint):
    return scipy.stats.multivariate_normal.pdf( ABpoint, mean = Gaussian2Mean, 
                                               cov = Gaussian2Covariance )

def GaussianMultiModal(ABpoint):
    return Gaussian1(ABpoint) + Gaussian2(ABpoint)

SimpleColorPlotFromFunc(GaussianMultiModal,
                       xmin=-1,
                       xmax=6,
                       ymin=-1,
                       ymax=6)

#### Use the emcee package and what you learned in the MCMC exercise to find the global peak of the joint probability density function `GaussianMultiModal`. Start with the initialized sampler below (with only 4 walkers and 125 steps per walker), but fill in a `lnprob` function that calculates the loglikelihood for a given choice of A and B. Note that in this case, the `GaussianMultiModal` function is exactly the likelihood.

In [ ]:
# Fill in your lnprob function here, it could also call a lnprior function that imposes bounds

In [ ]:
#Set up the MCMC to sample the full parameter space
ndim, nwalkers = 2, 4 #number of parameters to fit (3); number of individual "walkers" that randomly sample the space. Choose any number, the higher the slower.
best_guess_parameters = np.array([1,1])

# This sets the starting position of each walker. It's just creating a list
# of length nwalkers, and number of columns equal to ndim. Each row contains
# a random location for a walker to start, based on a Gaussian
# centered on your best guess. You can tighten or loosen this constraint by
# changing the "starting_location_width" parameter. Setting it to be a very small number
# like this is basically starting all walkers in the same location, which is the point of this 
# exercise.

starting_location_width = .0001
starting_positions = []
for i in range(nwalkers):
    starting_positions.append(best_guess_parameters + starting_location_width*np.random.randn(ndim))


In [ ]:
#set up the MCMC sampler
sampler = emcee.EnsembleSampler(nwalkers, 
                                ndim, 
                                lnprob) #the likelihood function to maximize
#run the MCMC with the starting positions we defined and 100 sampling points per walker.
#this is the equivalent of setting the number of steps each blindfolded person can 
#take in the room, using the example above.
n_samples_per_walker=125
output = sampler.run_mcmc(starting_positions, n_samples_per_walker) 

In [ ]:
#sampler.chain contains all of the samples from the MCMC.
print(sampler.chain.shape)
#It currently holds the samples separately for each walker.
#We don't care about what each walker does, so let's flatten it:
#The -1 here means we don't care how many rows it takes, 
#give us the same number of columns as we have parameters
samples = sampler.chain.reshape((-1, ndim)) 
print(samples.shape)
#So we tried 500 total model realizations for 3 parameters
LogProbabilities = sampler.lnprobability.reshape((-1, 1)).flatten()

In [ ]:
#So what did we actually get from this? Let's use another
#python package to see the output of the MCMC sampling.
#The dashed line is one way of estimating the best-fit parameters
#from an MCMC sampler (the median of the samples). The blue line
#shows the true values.

DomainBoxMinPoint=[0,-2]
DomainBoxMaxPoint=[4,5]

RangeObject = np.vstack([DomainBoxMinPoint, DomainBoxMaxPoint]).T
fig = corner.corner(samples, #samples is defined above
                    labels=["$A$", "$B$"],#parameter labels
                    truths=Gaussian2Mean,
                    plot_contours=False,plot_density=False,
                    plot_datapoints=True,range=RangeObject)

fig = corner.corner(samples, #samples is defined above
                    labels=["$A$", "$B$"],#parameter labels
                    truths=samples[LogProbabilities.argmax()],fig=fig,
                    plot_contours=False,plot_density=False,truth_color='red',
                    plot_datapoints=False,range=RangeObject)
plt.show()

In [ ]:
def cross_hair(x, y, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    horiz = ax.axhline(y,  **kwargs)
    vert = ax.axvline(x, **kwargs)
    return horiz, vert
best_guess_peak=samples[LogProbabilities.argmax()]

SimpleColorPlotFromFunc(GaussianMultiModal,
                       xmin=-1,
                       xmax=6,
                       ymin=-1,
                       ymax=6)
cross_hair(best_guess_peak[0],best_guess_peak[1],color='red')
print('Guess:',best_guess_peak,'Truth:',Gaussian2Mean)